In [69]:
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_target = df['SUBCLASS']

disease_list = df.SUBCLASS.unique()

dd_list = ['BRCA', 'KIPAN' , 'GBMLGG', 'STES']
#too many 변이, ROWs DELETE
for d in disease_list:
    td = df[df['SUBCLASS']==d].apply(lambda x : x!='WT').agg(sum , axis=1).sort_values(ascending=False) >= 800 #706
    if td[td].sum()> 0:
        print(d, list(td[td].index))
        df.drop(list(td[td].index), inplace=True)


data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns=['ID'], inplace=True)
data

SKCM [1369, 6153, 486]
STES [601, 682]
BRCA [4228, 2707]
PAAD [1789]
UCEC [4757, 1761, 4984, 2189, 5049, 1202, 6140, 4098, 5683, 406, 71, 520]
COAD [4081, 1502, 3967]


,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [70]:
df.shape

(6178, 4386)

In [71]:
data.shape

(6178, 4384)

In [84]:
def get_count_df(df, d, th):
    count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count').sort_values(by='count',ascending=False)
    gl = count_df[count_df['count']>th][1:]['index']
    return gl

from collections import defaultdict
expert_dict = defaultdict()

df_ = data.copy()

df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
for d in disease_list:
    all_g = set(df[df.SUBCLASS==d].columns)
    gl = get_count_df(df_, d, 1)
    try:
        g_expert = pd.read_csv('./reference_data/'+d+'.csv')
        aa = set(g_expert['g.name'])        
        
    except FileNotFoundError:
        aa = set(get_count_df(df_, d, 5))
        
    expdict = all_g.intersection(aa)
    expert_dict[d] = expdict
    print(d,  len(list(gl)), len(aa), len(expdict))

KIPAN 1873 699 307
SARC 513 69 38
SKCM 3407 69 31
KIRC 1050 132 69
GBMLGG 1131 181 117
STES 3566 69 38
BRCA 2369 1004 497
THCA 197 58 34
LIHC 1161 179 104
HNSC 1948 55 37
PAAD 362 628 287
OV 818 317 133
PRAD 370 611 311
UCEC 2203 89 54
LAML 49 1007 493
COAD 2519 557 278
ACC 555 38 23
LGG 288 111 53
LUSC 2399 629 246
LUAD 2116 467 467
CESC 1652 42 22
PCPG 72 3 3
THYM 30 19 10
BLCA 1774 121 70
TGCT 430 23 9
DLBC 553 9 4


In [73]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('disease').reset_index(name='counts')
diseaseList

,disease,counts
0,BRCA,784
1,KIPAN,515
2,GBMLGG,461
3,STES,377
4,KIRC,334
5,THCA,324
6,SKCM,273
7,PRAD,266
8,OV,253
9,LGG,229


In [81]:
class_dict = defaultdict()
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>5]['index'].values
        print(d, len(gene_list))
        class_dict[d] = gene_list
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

KIPAN 286
SARC 31
SKCM 1895
KIRC 68
GBMLGG 147
STES 1867
BRCA 548
THCA 4
LIHC 106
HNSC 334
PAAD 15
OV 47
PRAD 16
UCEC 402
LAML 9
COAD 610
ACC 51
LGG 16
LUSC 587
LUAD 468
CESC 175
PCPG 3
THYM 1
BLCA 209
TGCT 14
DLBC 17
2534


In [79]:
glist

{'GPRC5C',
 'METTL7B',
 'CSF2RB',
 'CYBB',
 'ADAM2',
 'UPF3B',
 'ZC3H12A',
 'DCTN1',
 'SULT2B1',
 'ORC2',
 'CXCR4',
 'IGF2R',
 'GLDC',
 'CTSS',
 'IGF1R',
 'ABCC8',
 'CDH16',
 'TNFSF11',
 'GEM',
 'IGSF3',
 'HGFAC',
 'NOD2',
 'ABL1',
 'PER1',
 'BCL11B',
 'VCAM1',
 'EIF3B',
 'NCAN',
 'FST',
 'PNPLA8',
 'MAP3K1',
 'RNMT',
 'KCNK5',
 'HMGCL',
 'SPI1',
 'TMEM176B',
 'ITGAE',
 'SLC6A12',
 'ALS2',
 'CEP72',
 'LAS1L',
 'ZBTB17',
 'CRYZ',
 'KCNMB2',
 'IL1RL1',
 'RAD21',
 'KRT31',
 'PPARGC1A',
 'BCL2L11',
 'MET',
 'ABCA5',
 'SFRP4',
 'MYO9B',
 'SYT12',
 'ATP7A',
 'TLE4',
 'DGKH',
 'EWSR1',
 'CACNA1F',
 'ATXN2',
 'SUCLG2',
 'SCAF4',
 'F13B',
 'HTR1D',
 'ECM1',
 'PIKFYVE',
 'ACACA',
 'FMO3',
 'WLS',
 'OSM',
 'SRC',
 'TFCP2L1',
 'AMOT',
 'TDO2',
 'PAXIP1',
 'RHOT1',
 'ITGB7',
 'MYH4',
 'ERCC4',
 'CYLD',
 'FGF9',
 'P2RY4',
 'GREB1',
 'CCNA1',
 'MKNK2',
 'BCAR1',
 'PKP2',
 'CROCC',
 'SGMS1',
 'FBLN5',
 'XYLT2',
 'FGF17',
 'APLP1',
 'BNIP3',
 'SUN2',
 'NCOA3',
 'EPHA5',
 'BCL6',
 'LTF',
 'TROAP',
 'STE

In [105]:
selected_genes = set()
for d in disease_list :
    expd = expert_dict[d]
    selected_genes.update(list(expd))
len(selected_genes)

2051

In [92]:
selected_genes = set()
for d in disease_list:
    tdf = df [ df['SUBCLASS'].isin([d] )]
    total_len = tdf.shape[0]
    
    tdf = tdf.apply(lambda x : x !='WT').agg(sum)
    tdf = tdf.iloc[2:].reset_index(name='count').sort_values(by='count', ascending=False)
    #thr[d]
    rlist =[]
    if total_len > 200: 
        rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()
    else :
        rlist = tdf.loc[tdf['count'] > 0]['index'].to_list()

    print(d, len(rlist))
    selected_genes.update(rlist)
#selected_genes = list(selected_genes)
#len(selected_genes)

KIPAN 286
SARC 1547
SKCM 1895
KIRC 68
GBMLGG 147
STES 1867
BRCA 548
THCA 4
LIHC 2394
HNSC 334
PAAD 1274
OV 47
PRAD 16
UCEC 3215
LAML 316
COAD 610
ACC 1580
LGG 16
LUSC 3330
LUAD 3123
CESC 2905
PCPG 524
THYM 388
BLCA 2931
TGCT 1366
DLBC 1657


In [106]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [6]:
#selected_genes.remove("SUBCLASS")

In [107]:
df_target = df['SUBCLASS']

data = data.loc[:, list(selected_genes)]
df_test = df_test.loc[:, list(selected_genes)]


In [29]:
df_target = df['SUBCLASS']

data = data.loc[:, list(glist)]
df_test = df_test.loc[:, list(glist)]


In [108]:
total = pd.concat([data, df_test] ) 
f = lambda x : '' if x == 'WT' else x
total = total.map(f)
total

,RPS14,LHX2,NIP7,BPHL,HEY1,CSF2RB,CYBB,ADAM2,SULT2B1,CXCR4,...,TPH1,NRIP1,PC,TMEM100,PDHB,CD44,KDELR3,ITK,MDM2,NBN
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,Q34Q,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,T176T,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,Y45C,,,,,,,,...,,E729Rfs,,L99Yfs,Q160H Q142H,,,K538E,A345T A175T A296T A149T A351T A298T,Y322X Y240X
2544,,,,,,,,,,,...,,,,,,,,,,


In [109]:
total = total.loc[:, list(selected_genes)]
total

,RPS14,LHX2,NIP7,BPHL,HEY1,CSF2RB,CYBB,ADAM2,SULT2B1,CXCR4,...,TPH1,NRIP1,PC,TMEM100,PDHB,CD44,KDELR3,ITK,MDM2,NBN
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,Q34Q,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,T176T,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,Y45C,,,,,,,,...,,E729Rfs,,L99Yfs,Q160H Q142H,,,K538E,A345T A175T A296T A149T A351T A298T,Y322X Y240X
2544,,,,,,,,,,,...,,,,,,,,,,


In [110]:
df.shape

(6178, 4386)

In [111]:
#df = total[:6201]
#df_test = total[6201:]

In [112]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
total = total.apply(label_encoder.fit_transform)

nRows = df.shape[0]
X = total[:nRows]
df_test = total[nRows:]
y = df_target



encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)
y


array([ 8, 19, 20, ...,  4, 22, 20])

In [113]:
y.shape[0]

6178

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)#shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4324, 2051) (1854, 2051)


#Multi-class classification

In [115]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [116]:
params = {
    'tree_method': 'approx',
    'objective':  'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.59801	valid-mlogloss:2.71834
[2]	train-mlogloss:2.20768	valid-mlogloss:2.48058
[4]	train-mlogloss:1.97825	valid-mlogloss:2.35415
[6]	train-mlogloss:1.82106	valid-mlogloss:2.28350
[8]	train-mlogloss:1.70391	valid-mlogloss:2.23629
[10]	train-mlogloss:1.61031	valid-mlogloss:2.20632
[12]	train-mlogloss:1.53453	valid-mlogloss:2.18569
[14]	train-mlogloss:1.47225	valid-mlogloss:2.17227
[16]	train-mlogloss:1.41951	valid-mlogloss:2.16097
[18]	train-mlogloss:1.37270	valid-mlogloss:2.15176
[20]	train-mlogloss:1.33184	valid-mlogloss:2.14648
[22]	train-mlogloss:1.29459	valid-mlogloss:2.14546
[24]	train-mlogloss:1.26198	valid-mlogloss:2.14691
[26]	train-mlogloss:1.23292	valid-mlogloss:2.14504
[28]	train-mlogloss:1.20556	valid-mlogloss:2.14307
[30]	train-mlogloss:1.17865	valid-mlogloss:2.14485
[32]	train-mlogloss:1.15426	valid-mlogloss:2.14465
[34]	train-mlogloss:1.12922	valid-mlogloss:2.14502
[36]	train-mlogloss:1.10768	valid-mlogloss:2.14527


In [117]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    #'learning_rate': 0.05
}
num_boost_round = 1000

clf = xgb.XGBClassifier( n_estimators = 100,        # 부스팅 단계 수
                        max_depth = 8,             # 개별 트리의 최대 깊이
                        learning_rate = 0.1,       # 학습률, 낮출수록 느리게 학습하지만 성능 개선 가능
                        subsample=0.8,           # 각 부스팅 단계에서 사용할 샘플의 비율
                        #scale_pos_weight = class_weight_dict,  # 클래스별 가중치
                        colsample_bytree=0.8,    # 각 트리에서 사용할 특성의 비율
                        random_state = 42,         # 랜덤 시드
                        n_jobs = -1,             # 모든 CPU 코어 사용,
                        early_stopping_rounds = 5,
                        **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.07264	validation_1-mlogloss:3.10968
[2]	validation_0-mlogloss:2.77256	validation_1-mlogloss:2.87291
[4]	validation_0-mlogloss:2.57143	validation_1-mlogloss:2.72386
[6]	validation_0-mlogloss:2.42171	validation_1-mlogloss:2.62716
[8]	validation_0-mlogloss:2.29702	validation_1-mlogloss:2.54738
[10]	validation_0-mlogloss:2.19126	validation_1-mlogloss:2.48625
[12]	validation_0-mlogloss:2.09892	validation_1-mlogloss:2.43342
[14]	validation_0-mlogloss:2.01939	validation_1-mlogloss:2.38938
[16]	validation_0-mlogloss:1.94804	validation_1-mlogloss:2.35301
[18]	validation_0-mlogloss:1.88403	validation_1-mlogloss:2.32236
[20]	validation_0-mlogloss:1.82531	validation_1-mlogloss:2.29547
[22]	validation_0-mlogloss:1.77325	validation_1-mlogloss:2.27331
[24]	validation_0-mlogloss:1.72434	validation_1-mlogloss:2.25363
[26]	validation_0-mlogloss:1.67961	validation_1-mlogloss:2.23745
[28]	validation_0-mlogloss:1.63929	validation_1-mlogloss:2.22190
[30]	validation_0-mlogloss:1.6

In [118]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1854,), (1854,), (1854, 26))

In [119]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.33063646170442285

In [41]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.33225458468176916

In [120]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.63      0.75        19
           1       0.36      0.11      0.17        36
           2       0.27      0.70      0.39       226
           3       0.15      0.06      0.09        48
           4       0.51      0.57      0.54        65
           5       0.25      0.12      0.17         8
           6       0.34      0.34      0.34       149
           7       0.33      0.15      0.21        78
           8       0.20      0.18      0.19       164
           9       0.16      0.16      0.16        86
          10       0.67      0.50      0.57        40
          11       0.22      0.19      0.21        77
          12       0.45      0.23      0.30        44
          13       0.30      0.17      0.22        52
          14       0.28      0.15      0.19        47
          15       0.22      0.47      0.30        68
          16       0.29      0.06      0.10        33
          17       0.19    

In [43]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8578955349853364

In [121]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8585661016212224

## Create Result DataFrame

In [122]:
clf.predict(df_test)

array([21, 25,  2, ..., 22, 10,  6])

In [123]:
res = clf.predict(df_test)
res = encoder2.inverse_transform(res)
df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,GBMLGG
4,TEST_0004,LUSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,TGCT
2544,TEST_2544,LAML


In [124]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      631
COAD      476
KIPAN     213
STES      175
OV        144
GBMLGG    132
SKCM      100
THCA       77
UCEC       66
KIRC       64
TGCT       56
LGG        51
HNSC       49
LAML       40
LIHC       40
LUAD       37
CESC       35
PRAD       33
PCPG       28
SARC       23
BLCA       21
ACC        21
LUSC       18
PAAD        9
DLBC        5
THYM        2
Name: count, dtype: int64

In [125]:
res_df.to_csv('res3-0.csv',index = False)

## 01 Update Result with Zeros []

In [53]:
zerosl = list(df_test[(df_test == 0).all(axis=1)].index)
res_df.loc[zerosl, 'SUBCLASS'] = 'THYM'

In [54]:
res_df

,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,GBMLGG
4,TEST_0004,LUSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,SKCM
2544,TEST_2544,LAML


In [55]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      598
COAD      493
KIPAN     215
STES      159
OV        142
GBMLGG    137
SKCM       85
THCA       73
KIRC       73
UCEC       60
HNSC       56
LGG        51
TGCT       49
LIHC       48
LAML       44
THYM       38
LUAD       34
CESC       33
PCPG       30
PRAD       28
SARC       23
BLCA       23
ACC        20
LUSC       15
PAAD       13
DLBC        6
Name: count, dtype: int64

In [56]:
res_df.loc[180]

ID          TEST_0180
SUBCLASS         THYM
Name: 180, dtype: object

## SAVE res_df with zero step

In [57]:
res_df.to_csv('res2-step-1.csv',index = False)

In [58]:
############## <5 Finding SubSet

In [59]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns='ID',inplace=True)

total = pd.concat([data, df_test])

f = lambda x : 0 if x == 'WT' else 1
total = total.map(f)

data = total[:6201]
df_test = total[6201:]


In [60]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#하나라도 1인  

# 딱 1개만 1인 행 필터링
one_1_rows = df_test[df_test.sum(axis=1) == 1]

# 각 행에서 1이 위치한 열의 이름과 인덱스를 구하는 코드
result = [(index, row.idxmax()) for index, row in one_1_rows.iterrows()]
result

[(8, 'SOX9'),
 (90, 'BRAF'),
 (110, 'LIG1'),
 (131, 'IDH2'),
 (170, 'PLK4'),
 (192, 'ZNF365'),
 (260, 'BRAF'),
 (352, 'PRKDC'),
 (366, 'VHL'),
 (390, 'DOCK2'),
 (435, 'CDCP1'),
 (619, 'GFPT2'),
 (655, 'CTNNA1'),
 (672, 'CD46'),
 (693, 'HRAS'),
 (701, 'HRAS'),
 (798, 'CCNL1'),
 (880, 'GNAO1'),
 (898, 'ATXN1'),
 (1023, 'MYH1'),
 (1034, 'CDKN2A'),
 (1071, 'VHL'),
 (1078, 'CACNA1A'),
 (1118, 'ACE'),
 (1202, 'VHL'),
 (1263, 'BRAF'),
 (1287, 'TG'),
 (1408, 'VHL'),
 (1426, 'VHL'),
 (1437, 'SREBF2'),
 (1560, 'CACNA1A'),
 (1690, 'LRPPRC'),
 (1761, 'VHL'),
 (1804, 'RAD21'),
 (1830, 'LIG1'),
 (1833, 'VHL'),
 (1851, 'NPM1'),
 (1879, 'VHL'),
 (1884, 'KMT2D'),
 (1979, 'NPM1'),
 (2056, 'VHL'),
 (2086, 'VHL'),
 (2114, 'BRCA2'),
 (2150, 'NPM1'),
 (2218, 'TP53'),
 (2383, 'NPM1'),
 (2447, 'CDH1'),
 (2512, 'SUCLG1')]

In [ ]:
one_VHL_l=[]
one_BRAF_l = []
one_NPM1_l = []
one_LIG1_l = []
one_s = []
res_df1 = res_df.copy()

for i, c in result:
    print(i, c)
    if c =='VHL': 
        one_VHL_l.append(i)
        # 해당 컬럼에서 SUBCLASS값들의 분포를 구함
        q = ['VHL']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)] 
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #KIRC
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
    elif c=='BRAF': 
        one_BRAF_l.append(i)
        q = ['BRAF']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #maybe 1
        res_df1.loc[one_BRAF_l,'SUBCLASS'] = rs
    elif c=='NPM1' : 
        one_NPM1_l.append(i)
        q = ['NPM1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_NPM1_l,'SUBCLASS'] = rs
    elif c=='LIG1' : 
        one_LIG1_l.append(i)
        q = ['LIG1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_LIG1_l,'SUBCLASS'] = rs
    elif c=='HRAS': 
        one_s.append(i)
        q = ['HRAS']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS#0 not 1
        res_df1.loc[one_s,'SUBCLASS'] = rs
    #else:
        ###one_s.append(i)
        #row_no = i
        q = [c]
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
        res_df1.loc[i]['SUBCLASS'] = rs
    

res_df1.to_csv('res-step2-ones-2-else2.csv',index = False)
    

In [ ]:
############# optional code

In [ ]:
## TEST 데이터에서 1개의 컬럼만 1인 경우 찾기
one_1_rows = df_test[df_test.sum(axis=1) == 1]
# 각 열(column)별로 1이 있는 인덱스 리스트 생성
result_dict = {}
for column in one_1_rows.columns:
    indices = one_1_rows[one_1_rows[column] == 1].index.tolist()
    if indices:  # 인덱스가 비어있지 않으면 추가
        result_dict[column] = indices

# 딕셔너리를 DataFrame으로 변환
result_df = pd.DataFrame(list(result_dict.items()), columns=['gene', 'index_list'])
result_df


In [ ]:
for q in result_df.gene:
    print(q)
    rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
    rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    if rs.shape[0] > 1:
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs.loc[1].SUBCLASS
    elif rs.shape[0] ==1:
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs.loc[0].SUBCLASS
    print(rs)
res_df1.to_csv('res-step2-ones-3.csv',index = False)

In [ ]:
q = 'VHL'
rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
#res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
print(rs)

In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## 1이 2개 이상 있는 행 필터링
one_2_rows = df_test[df_test.sum(axis=1) == 2]

# 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 출력
print(result)


## Update Res_df with 2 / 3 / 4 / 5

In [ ]:
data = data.loc[:, list(glist)]
df_test = df_test.loc[:, list(glist)]

In [63]:
## TEST DATA에서 1이 n개 이상 있는 행 필터링
def find_test_n_ones(n):
    one_2_rows = df_test[df_test.sum(axis=1) == n]
    # 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
    result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]
    return result
print(find_test_n_ones(3))

[(11, ['HK1', 'NPM1', 'VIPR1']), (22, ['ALMS1', 'ATRX', 'IDH1']), (145, ['BRAF', 'CDC42EP4', 'PPRC1']), (241, ['NCOA6', 'PABPC1', 'RGN']), (320, ['AIFM1', 'THRB', 'TUBGCP6']), (333, ['MMP2', 'PTEN', 'TRIM58']), (398, ['ABCG4', 'LCT', 'SCN9A']), (456, ['IDH2', 'NOTCH1', 'RUNX1']), (490, ['BRAF', 'FETUB', 'LAMC1']), (491, ['GATA3', 'NRCAM', 'PLCG1']), (492, ['HSP90AB1', 'MKNK1', 'PTEN']), (586, ['BRAF', 'GOT2', 'UQCRC1']), (656, ['GPD2', 'HRAS', 'PUS1']), (669, ['ANXA5', 'MKI67', 'RET']), (736, ['CCND3', 'CD59', 'HNF4A']), (812, ['AHNAK', 'DNAJB8', 'SUN2']), (852, ['NRIP1', 'PIK3CA', 'SLC30A1']), (856, ['ATRN', 'FLII', 'PLCG1']), (897, ['FANCC', 'SFRP1', 'SPTB']), (935, ['AZGP1', 'BRAF', 'THBS1']), (942, ['BUB1', 'HAO2', 'PTPRE']), (954, ['KRT13', 'NPM1', 'RREB1']), (978, ['ARFGEF1', 'FZD5', 'TP53']), (1016, ['AP3B1', 'IRS1', 'TMPRSS2']), (1022, ['GAD1', 'IMMT', 'PCYT2']), (1089, ['ATRX', 'CA12', 'GAA']), (1102, ['CREBBP', 'MYBPC3', 'SPTA1']), (1139, ['CHEK2', 'PIK3CG', 'TAF1C']), (1197,

In [68]:
for ns in range (2,5):
    result_n = find_test_n_ones(ns )

    for i, qSet in result_n:
        #print(i, qSet)
        #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
        rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
        rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
        
        if rs.shape[0] > 0:
            rs = rs.loc[0].SUBCLASS
            print(rs, res_df.loc[i]['SUBCLASS'])
            #res_df2.loc[i,'SUBCLASS'] = rs
    


LGG BRCA
LGG GBMLGG
GBMLGG GBMLGG
KIRC UCEC
KIPAN PCPG
KIPAN SARC
LGG LGG
GBMLGG GBMLGG
GBMLGG GBMLGG
LGG LGG
LGG LGG
GBMLGG LGG
LGG LAML
GBMLGG GBMLGG
LGG LGG
LGG GBMLGG
LGG LGG
LGG LGG
GBMLGG GBMLGG


In [ ]:
print(find_test_n_ones(4))

In [ ]:
print(find_test_n_ones(5))


In [ ]:
df_test

In [ ]:
# res 값 수정, 
# 문서 2,3,4,5
res_df2 = res_df1.copy()
result_n = find_test_n_ones(2)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1:
            try:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count')#.loc[].SUBCLASS## 0->1
                print(rs)
                #res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS## 0->1
                res_df2.loc[i,'SUBCLASS'] = rs
                print(rs)
            
#res_df2.to_csv('res2-step3-two.csv',index = False)

In [ ]:
result_n = find_test_n_ones(3)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    #else: 
    #    #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
    #    rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
    #    if rows_with_all_ones.shape[0] > 0 : 
    #        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
    #        print(rs)
    #        res_df2.loc[i,'SUBCLASS'] = rs


In [55]:
res_df2.to_csv('res-step4-three.csv',index = False)

NameError: name 'find_test_n_ones' is not defined

In [48]:
res_df2.to_csv('res-step5-four.csv',index = False)

In [ ]:
result_n = find_test_n_ones(5)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1 : 
            try: 
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs


In [58]:
res_df2.to_csv('res-step6-five.csv',index = False)